# Parking Problem, and Eating a Magic Cake

## Andres Perez Mtz (worked in a group)
(7.5/5) 2.5 extra points

### Question 1
Solve the parking problem from the midterm in fewer than 20 lines of code.  (I did it in six; anyone coming in with ten or fewer lines will get 2.5 points extra credit.). You can truncate the state space to go from -100 to +100.  Don't forget: the state is something like (-13, open) or (-13, no_spot)

In [1]:
!pip install --upgrade pip
!pip install interpolation

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
import quantecon as qe
from interpolation import interp
from scipy.optimize import minimizer_scalar, bisect
from quantecon.optimize import brentq
from numba import njit, float64
from numba.experimental import jitclass

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'd:\\applications\\python\\scripts\\pip.exe'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:


Script file 'D:\Applications\Python\Scripts\pip-script.py' is not present.


ModuleNotFoundError: No module named 'interpolation'

In [ ]:
n, no_spot, p, max_iter = 100,-500,0.20,1000
V = [-abs(np.arange(-n,n+1)),no_spot*np.ones(2*n+1)]

for i in range(max_iter):
    V[0][:n],V[1][:-1] =np.maximum(V[0][:n],p*V[0][1:n+1]+(1-p)*V[1][1:n+1]), p*V[0][1:]+(1-p)*V[1][1:]
print("Park Here or Next Spot:",V[0][0])

### Question 2
You have this utility function:

$$E\left[\sum_{t=0}^\infty \beta^tu(c_t)\right] $$

and a nice genie has given you a magic cake.  Its maximal size is 10 .  You get to eat a piece of any size from the cake.  The genie waves her magic wand over night, and she shocks the cake with a multiplicative random variable 
whose logarithm is independently and identically distributed $N(\mu,\sigma^2)$. Your goal is to maximize expected utility.

Describe this problem as a dynamic program:

What is the state?
What is the control?
What is the law of motion?
What is the reward?
What is the discount factor?
Write Bellman's equation for this problem.

Assume that 

$$u(c)= \frac{c^{1-r}-1}{1-r}$$ 

for $r \ge 0$. Fix $r = 2$, $\beta$ = 0.97, $\mu=-0.005$, and $\sigma=0.1$. Using np.random.seed(1234), generate a fixed sequence of 250 log normally distributed random variables. Use what Sargent calls time iteration to solve this dynamic program.  You can work together.  Please indicate joint submissions clearly.  You will find a lot of code on Sargent's web page that will help you with aspects of this problem.  Here are some hints:  (1) generate a fixed sequence of shocks; (2) use the method euler_diff from the lecture on Coleman's method, adapted to the cake eating problem; and (3) you will need to use Monte Carlo integration to compute the expected marginal felicity from consuming next period's cake.  My algorithm converged after 626 iterations; so you may have to increase max_iter.

 



In [ ]:
def maximize(g, a, b, args):
    """
    Maximize the function g over the interval [a, b].

    We use the fact that the maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.

    Returns the maximal value and the maximizer.
    """

    objective = lambda x: -g(x, *args)
    result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum

In [ ]:
def K(σ_array, ce):
    """
    The policy function operator. Given the policy function,
    it updates the optimal consumption using Euler equation.

    * σ_array is an array of policy function values on the grid
    * ce is an instance of CakeEating

    """

    u_prime, β, x_grid = ce.u_prime, ce.β, ce.x_grid
    σ_new = np.empty_like(σ_array)
    
    seed = 1234
    np.random.seed(seed)
    shock_size = 250 
    μ = -0.005
    s = 0.1
    shocks = np.exp(μ + s * np.random.randn(shock_size))
    
    #σ = lambda x: interp(x_grid, σ_array, x)

    def euler_diff(c, x):
        σ_func = lambda x: interp(x_grid, σ_array, x)
        vals = u_prime(σ_func((x-c) * shocks))
        return u_prime(c) - β * np.mean(vals)

    for i, x in enumerate(x_grid):

        # handle small x separately --- helps numerical stability
        if x < 1e-12:
            σ_new[i] = 0.0

        # handle other x
        else:
            σ_new[i] = bisect(euler_diff, 1e-10, x - 1e-10, x)

    return σ_new

In [ ]:
class CakeEating:

    def __init__(self,
                 β=0.97,           # discount factor
                 γ=2,            # degree of relative risk aversion
                 x_grid_min=1e-3,  # exclude zero for numerical stability
                 x_grid_max=10,   # size of cake
                 x_grid_size=120):

        self.β, self.γ = β, γ

        # Set up grid
        self.x_grid = np.linspace(x_grid_min, x_grid_max, x_grid_size)

    # Utility function
    def u(self, c):

        γ = self.γ

        if γ == 1:
            return np.log(c)
        else:
            return (c ** (1 - γ)) / (1 - γ)

    # first derivative of utility function
    def u_prime(self, c):

        return c ** (-self.γ)

    def state_action_value(self, c, x, v_array):
        """
        Right hand side of the Bellman equation given x and c.
        """

        u, β = self.u, self.β
        v = lambda x: interp(self.x_grid, v_array, x)

        return u(c) + β * v(x - c)

In [ ]:
def iterate_euler_equation(ce,
                           max_iter=5000,
                           tol=1e-5,
                           verbose=True,
                           print_skip=25):

    x_grid = ce.x_grid

    σ = np.copy(x_grid)        # initial guess

    i = 0
    error = tol + 1
    while i < max_iter and error > tol:

        σ_new = K(σ, ce)

        error = np.max(np.abs(σ_new - σ))
        i += 1

        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")

        σ = σ_new

    if error > tol:
        print("Failed to converge!")
    elif verbose:
        print(f"\nConverged in {i} iterations.")

    return σ

In [ ]:
ce = CakeEating(x_grid_min = 0.0)
c_euler = iterate_euler_equation(ce)

In [ ]:
def c_star(x, β, γ):
    
    return(1 - β **(1/γ)) * x

c_analytical = c_star(ce.x_grid, ce.β, ce.γ)

In [ ]:
fig, ax = plt.subplots()

ax.plot(ce.x_grid, c_analytical, label='Analytical Solution')
ax.plot(ce.x_grid, c_euler, label='Time Iteration Solution')

ax.set_ylabel('Sonsumption')
ax.set_xlabel('$X$')
ax.legend(fontsize=12)

plt.show()